In [ ]:
# Block 0: Imports

In [ ]:
# Block 1: Initialize Data directory & instructions for project setup / organization

In [ ]:
# Block 2: Convert downloaded PlanetScope RGB directories to RGB JPEGs to be labeled

In [ ]:
# Block 3: Download all mask batches from NDJSON files

In [ ]:
# Block 4: Process masks - clip to TIFs & create color masks if requested

In [ ]:
# Block 5: Download SR TIFs from each RGB file to be processed for training

In [ ]:
# Block 6: Train on masks & data. Show plots from classification

In [ ]:
# Block 7: Download batches of images from desired study sites en masse

In [ ]:
# Block 8: Test classification scores / Validate training

In [ ]:
# Block 9: Create time series for analysis